# 分析

本当は1日単位でDailyでとって変遷まで見たかったが、
間に合わないのでWeeklyで取るしかない。


## speed_car_10_10_10_16

### 文字数

#### Slack

下処理について

* 不要なjsonデータは削除しました
  * 主にトップフォルダの内容


In [30]:
import os
import json

# JSONファイルが格納されているディレクトリのパス
directory = 'material/speed_car_10_10_10_16/Slack/'

# 文字数の合計を格納する変数
total_characters = {}

# ディレクトリとそのサブディレクトリ内のすべてのファイルを読み込む
for subdir, dirs, files in os.walk(directory):
    for filename in files:
        if filename.endswith(".json"): 
            filepath = subdir + os.sep + filename
            with open(filepath, 'r') as f:
                data = json.load(f)
                
                for message in data:
                    if 'blocks' in message and 'user_profile' in message:
                       author = message['user_profile']['first_name']
                       if author not in total_characters:
                            total_characters[author] = 0
                       for block in message['blocks']:
                          if 'elements' in block:
                            for element in block['elements']:
                              if 'rich_text_section' in element['type']:
                                for element2 in element['elements']:
                                  if 'text' == element2['type']:
                                    text = element2['text']
                                    total_characters[author] += len(text)                                    
                                    
print(f"Total number of characters: {total_characters}")

print(total_characters['Nakayama'])
print(total_characters['satoshi.\u200bhirata'])
print(total_characters['Hibiki'])
print(total_characters['鈴木真希'])
print(total_characters['Dai'])
print(total_characters['Ohno'])
print(total_characters['FABIAN'])


Total number of characters: {'satoshi.\u200bhirata': 2517, 'Hibiki': 3245, 'Dai': 1566, 'Nakayama': 1405, 'Ohno': 2310, '鈴木真希': 881, 'FABIAN': 463}
1405
2517
3245
881
1566
2310
463


#### Figjam


In [31]:
import pandas as pd

# CSVファイルのパス
filepath = 'material/speed_car_10_10_10_16/figjam.csv'

# CSVファイルを読み込む
df = pd.read_csv(filepath)

# 'Content'列の文字数をカウントする
df['char_count'] = df['Content'].str.len()

# 'Created by'でグループ化し、各グループの文字数の合計を計算する
char_count_by_creator = df.groupby('Created by')['char_count'].sum()

print(char_count_by_creator)

print("Total: " + str(char_count_by_creator.sum()))

print(char_count_by_creator['中山建太'])
print(char_count_by_creator['平田聖'])
print(char_count_by_creator['水野響'])
print(char_count_by_creator['鈴木真希'])
print(char_count_by_creator['Dai Miyahara'])
print(char_count_by_creator['大野寛人'])
print(char_count_by_creator['FABIAN M. FERNANDEZ'])



Created by
Dai Miyahara           270
FABIAN M. FERNANDEZ    112
中山建太                   367
大野寛人                   299
平田聖                    184
水野響                    276
鈴木真希                    74
Name: char_count, dtype: int64
Total: 1582
367
184
276
74
270
299
112


### 文章量

#### Slack

In [3]:
import os
import json
import MeCab
import ipadic

CHASEN_ARGS = r' -F "%m\t%f[7]\t%f[6]\t%F-[0,1,2,3]\t%f[4]\t%f[5]\n"'
CHASEN_ARGS += r' -U "%m\t%m\t%m\t%F-[0,1,2,3]\t\t\n"'

# JSONファイルが格納されているディレクトリのパス
directory = 'material/speed_car_10_10_10_16/'

# MeCabのインスタンスを作成
m = MeCab.Tagger(ipadic.MECAB_ARGS + CHASEN_ARGS)

# 文章数と単語数の合計を格納する変数
total_sentences = 0
total_words = 0

# ディレクトリとそのサブディレクトリ内のすべてのファイルを読み込む
for subdir, dirs, files in os.walk(directory):
    for filename in files:
        if filename.endswith(".json"): 
            filepath = subdir + os.sep + filename
            with open(filepath, 'r') as f:
                data = json.load(f)
                for message in data:
                    if 'text' in message:
                        text = message['text']
                        total_sentences += text.count('.') + text.count('!') + text.count('?') + text.count('\n') + text.count(',')
                        total_words += len(m.parse(text).split())

print(f"Total number of sentences: {total_sentences}")
print(f"Total number of words: {total_words}")

Total number of sentences: 998
Total number of words: 64026


#### Figjam

In [5]:
import pandas as pd
import MeCab

# CSVファイルのパス
filepath = 'material/speed_car_10_10_10_16/figjam.csv'

# CSVファイルを読み込む
df = pd.read_csv(filepath)

# MeCabの初期化
m = MeCab.Tagger(ipadic.MECAB_ARGS + CHASEN_ARGS)

# 'Content'列の各行を単語に分割し、単語の数をカウントする
df['word_count'] = df['Content'].apply(lambda x: len(m.parse(x).split()))

# 単語の合計数を計算する
total_word_count = df['word_count'].sum()

# 'Content'列の行数（文章の数）をカウントする
total_sentence_count = df['Content'].count()

print(f"Total number of words: {total_word_count}")
print(f"Total number of sentences: {total_sentence_count}")

Total number of words: 3639
Total number of sentences: 58


### 単語のカウント数


#### Slack

In [18]:
import os
import json
import MeCab
import ipadic
from collections import Counter
import re

CHASEN_ARGS = r' -F "%m\t%f[7]\t%f[6]\t%F-[0,1,2,3]\t%f[4]\t%f[5]\n"'
CHASEN_ARGS += r' -U "%m\t%m\t%m\t%F-[0,1,2,3]\t\t\n"'

# JSONファイルが格納されているディレクトリのパス
directory = 'material/speed_car_10_10_10_16/'

# MeCabのインスタンスを作成
m = MeCab.Tagger(ipadic.MECAB_ARGS + CHASEN_ARGS)

# 単語のカウントを格納するCounterオブジェクト
word_counts = Counter()

# ディレクトリとそのサブディレクトリ内のすべてのファイルを読み込む
for subdir, dirs, files in os.walk(directory):
    for filename in files:
        if filename.endswith(".json"): 
            filepath = subdir + os.sep + filename
            with open(filepath, 'r') as f:
                data = json.load(f)
                for message in data:
                    if 'text' in message:
                        text = message['text']
                        # HTMLエンティティを除外
                        text = re.sub(r'&[a-z]+;', '', text)
                        text = re.sub(r'https*:[a-zA-Z0-9%./?]+', '', text)
                        text = re.sub(r'AT-[0-9A-Za-z]+', '', text)
                        text = re.sub(r'[0-9]+', '', text)
                        parsed_text = m.parse(text).split('\n')
                        words = [word.split('\t')[0] for word in parsed_text 
                                 if word.strip() 
                                 and len(word.split('\t')) > 1 
                                 and (
                                     word.split('\t')[3].startswith('名詞') 
                                     or word.split('\t')[3].startswith('動詞') 
                                     or word.split('\t')[3].startswith('形容詞')
                                 ) 
                                 and len(word.split('\t')[0]) > 1 
                                 ]  
                        # 名詞、動詞、形容詞のみを抽出し、一文字の単語や英字のみの単語を除外
                        word_counts.update(words)

# 最も頻繁に出現する単語の上位3つを出力
top_three = word_counts.most_common(20)
for word, count in top_three:
    print(f'Word: {word}, Count: {count}')

Word: する, Count: 93
Word: from, Count: 68
Word: 作成, Count: 64
Word: 所持, Count: 61
Word: pt, Count: 56
Word: git, Count: 48
Word: atomono, Count: 43
Word: transitioned, Count: 42
Word: front, Count: 41
Word: 不在, Count: 39
Word: Jabelic, Count: 38
Word: In, Count: 38
Word: FE, Count: 36
Word: こと, Count: 35
Word: ブランチ, Count: 35
Word: リスト, Count: 34
Word: UTTDBW, Count: 34
Word: pull, Count: 33
Word: Hibiki, Count: 32
Word: Mizuno, Count: 32


#### Figjam

In [18]:
import pandas as pd
import MeCab
import ipadic
from collections import Counter
import re

CHASEN_ARGS = r' -F "%m\t%f[7]\t%f[6]\t%F-[0,1,2,3]\t%f[4]\t%f[5]\n"'
CHASEN_ARGS += r' -U "%m\t%m\t%m\t%F-[0,1,2,3]\t\t\n"'

# CSVファイルのパス
filepath = 'material/speed_car_10_10_10_16/figjam.csv'

# CSVファイルを読み込む
df = pd.read_csv(filepath)

# MeCabのインスタンスを作成
m = MeCab.Tagger(ipadic.MECAB_ARGS + CHASEN_ARGS)

# 単語をカウントする関数を定義
def count_words(text):
    if not isinstance(text, str):
        return []
    parsed_text = m.parse(text).splitlines()[:-1]
    words = []
    for word in parsed_text:
        word_parts = word.split('\t')
        if len(word_parts) >= 4 and not (word_parts[3].startswith('助詞') or word_parts[3].startswith('記号')):
            # Remove symbols from the word
            word_clean = re.sub(r'\W+', '', word_parts[0])
            if word_clean:
                words.append(word_clean)
    return words

# 'Content'列に関数を適用して新しい列を作成
df['Content_words'] = df['Content'].apply(count_words)

# 'Created by'でグループ化し、各グループの単語を集計
words_by_creator = df.groupby('Created by')['Content_words'].sum()

# 各作成者の最も頻繁に出現する単語の上位3つを計算
for creator, words in words_by_creator.items():
    word_counts = Counter(words)
    top_three = word_counts.most_common(10)
    print(f'Creator: {creator}')
    for word, count in top_three:
        print(f'    Word: {word}, Count: {count}')

Creator: Dai Miyahara
    Word: ゴール, Count: 4
    Word: する, Count: 3
    Word: 意識, Count: 3
    Word: 各自, Count: 2
    Word: ない, Count: 2
    Word: 的, Count: 2
    Word: な, Count: 2
    Word: Slack, Count: 2
    Word: し, Count: 2
    Word: 先, Count: 2
Creator: FABIAN M. FERNANDEZ
    Word: た, Count: 2
    Word: 良い, Count: 2
    Word: デモ, Count: 2
    Word: 情報, Count: 1
    Word: 共有, Count: 1
    Word: やり方, Count: 1
    Word: 何とか, Count: 1
    Word: 調べ, Count: 1
    Word: 本当に, Count: 1
    Word: これ, Count: 1
Creator: 中山建太
    Word: する, Count: 3
    Word: f, Count: 3
    Word: スキル, Count: 2
    Word: な, Count: 2
    Word: 進め方, Count: 2
    Word: 難しい, Count: 2
    Word: 作業, Count: 2
    Word: の, Count: 2
    Word: 活用, Count: 2
    Word: Q, Count: 2
Creator: 大野寛人
    Word: ない, Count: 4
    Word: 上, Count: 3
    Word: slack, Count: 3
    Word: 良い, Count: 3
    Word: 時間, Count: 2
    Word: QA, Count: 2
    Word: の, Count: 2
    Word: 思う, Count: 2
    Word: やっ, Count: 2
    Word: 人, Count: 2
